In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
import IPython.display
import tensorflow as tf

In [3]:

data_path = './data/'
training_number = 40
item_num = 50
nb_classes = 10
test_num = 10

x_data_raw = [None] * nb_classes
y_data_raw = [None] * nb_classes
for sub_path in range(nb_classes):
    x_data_raw[sub_path] = []
    y_data_raw[sub_path] = []
    for i in range(item_num):
        filename = data_path + str(sub_path) + '/' + "input" + str(i + 1) + '.wav'
        if os.path.exists(filename):
            raw, sr = librosa.load(filename, sr=None)
            x_data_raw[sub_path].append(librosa.feature.mfcc(y=raw, sr=sr))
            y_data_raw[sub_path].append(sub_path)
    print(np.shape(x_data_raw[sub_path]))

(50, 20, 862)
(50, 20, 862)
(50, 20, 862)
(50, 20, 862)
(50, 20, 862)
(50, 20, 862)
(50, 20, 862)
(50, 20, 862)
(50, 20, 862)
(50, 20, 862)


In [4]:

#96 X 431 사이즈
tf.InteractiveSession().as_default()
tf.tables_initializer().run()

#train, test set 분할
x_data = np.array(x_data_raw)
y_data = np.array(y_data_raw)
x_train = []
y_train = []
x_test = []
y_test = []

print(np.shape(x_data))
print(np.shape(y_data))

(10, 50, 20, 862)
(10, 50)


In [5]:

for i in range(nb_classes):
    x_train.append(x_data[i][:training_number])
    y_train.append(y_data[i][:training_number])
    x_test.append(x_data[i][training_number:training_number+test_num])
    y_test.append(y_data[i][training_number:training_number+test_num])

print(np.shape(x_train))
print(np.shape(y_train))
print(np.shape(x_test))
print(np.shape(y_test))

(10, 40, 20, 862)
(10, 40)
(10, 10, 20, 862)
(10, 10)


In [6]:
x_train = np.reshape(x_train, (-1, 20, 862))
x_test = np.reshape(x_test, (-1, 20, 862))

print(np.shape(x_train))
print(np.shape(x_test))


(400, 20, 862)
(100, 20, 862)


In [10]:
x_train = np.reshape(x_train, (-1, 100))
x_test = np.reshape(x_test, (-1, 100))

print(np.shape(x_train))
print(np.shape(x_test))

(68960, 100)
(17240, 100)


In [3]:
# FCN

training_number = 18000
learning_rate = 0.0001


with tf.variable_scope("sound_classifications-1-3") as scope:
    tf.variable_scope(scope, reuse=True)
    
    X = tf.placeholder(tf.float32, [None, 20])
    Y = tf.placeholder(tf.float32, [None, nb_classes])

    W1 = tf.get_variable("W1", shape=[20, 500], initializer=tf.contrib.layers.xavier_initializer())
    b1 = tf.Variable(tf.random_normal([500]))
    L1 = tf.nn.relu(tf.matmul(X, W1) + b1)

    W2 = tf.get_variable("W2", shape=[500, 500], initializer=tf.contrib.layers.xavier_initializer())
    b2 = tf.Variable(tf.random_normal([500]))
    L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)


    W3 = tf.get_variable("W3", shape=[500, nb_classes], initializer=tf.contrib.layers.xavier_initializer())
    b3 = tf.Variable(tf.random_normal([nb_classes]))
    hypothesis = tf.matmul(L2, W3) + b3

In [4]:
# Cross entropy cost/loss
#cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=hypothesis))

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Accuracy
is_correct = tf.equal(tf.arg_max(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Instructions for updating:
Use `argmax` instead


In [5]:
# Launch graph

batch_size = 200

cost_history = []
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for epoch in range(int(training_number/batch_size)):
        batch_count = int(x_data.shape[0]/batch_size)
        
        for i in range(batch_count):
    
            batch_xs, batch_ys = x_data[i* batch_size: i*batch_size+batch_size], y_data_one_hot[i*batch_size:i*batch_size+batch_size]
        
            sess.run(optimizer, feed_dict={X: batch_xs, Y: batch_ys})

            curr_cost = sess.run(cost, feed_dict={X: batch_xs, Y: batch_ys})
            
            
        if epoch % 10 == 0:    
            print(epoch, curr_cost)
            cost_history.append(curr_cost)
                                
                                
    print("Accuracy: ", accuracy.eval(session=sess, feed_dict={X: x_data, Y: y_data_one_hot}))


AttributeError: 'list' object has no attribute 'shape'